In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os

In [4]:
os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
spark = SparkSession.builder.appName('Premium Accounts').master('local').getOrCreate()

In [ ]:
top_sales_df = df_sales.alias('s').join(
        df_product.alias('p'),
        F.col('s.ProductID') == F.col('p.ProductID'),
        how = 'inner'
        ).withColumn(
            'SaleRank',
            F.rank().over (Window.partitionBy('s.CustomerID').orderBy(F.col('s.SaleDate').desc()))
            ).filter(
                (F.col('s.IsDeleted') == 0) & (F.col('SaleRank') == 1)
                ).select(
                    's.CustomerID',
                    's.ProductID',
                    'p.ProductName',
                    's.SaleDate',
                    's.Amount'
                    )
                
Customer_agg_df = df_customers.alias('c').join(
    df_sales.alias('s'),
    F.col('c.CustomerID') == F.col('s.CustomerID'),
    how = 'inner'
    ).filter(
        (F.col('s.SaleDate') >= F.add_months(F.current_date(), -6)) & (F.col('s.SaleDate') <= F.current_date())
        ).groupBy(
            'c.CustomerID',
            'c.CustomerName'
            ).agg(
                F.countDistinct('s.ProductID').alias('DistinctProducts'),
                F.sum('s.Amount').alias('TotalSpent'),
                F.max('s.SaleDate').alias('LastPurchaseDate')
                )
            
result = Customer_agg_df.alias('ca').join(
    top_sales_df.alias('ts'),
    F.col('ca.CustomerID') == F.col('ts.CustomerID'),
    how = 'left'
    ).filter(
        F.col('ca.DistinctProducts') >= 3
        ).select(
            'ca.CustomerID',
            'ca.CustomerName',
            'ca.DistinctProducts',
            'ca.TotalSpent',
            'ca.LastPurchaseDate',
            F.col('ts.ProductName').alias('LastPurchasedProduct'),
            F.when(
                F.col('ca.TotalSpent') > 10000, 'High Value'
                )
            .when(
                (F.col('ca.TotalSpent') >=  5000) & (F.col('ca.TotalSpent') <= 10000), 'Medium Value'
                )
            .otherwise('Low Value').alias('CustomerSegment')
            ).orderBy(
                F.col('ca.TotalSpent').desc()
                )

In [ ]:
CustomerActivity_df = df_customers.alias('c').join(
    df_orders.alias('o'),
    F.col('c.CustomerID') == F.col('o.CustomerID')
    ).withColumn(
        'OrderRank',
        F.rank().over(Window.partitionBy('c.CustomerID')).orderBy(F.col('o.OrderDate').desc())
        ).filter(
            (F.col('o.Status') == 'Completed') & (F.col('OrderRank') <= 3)
            ).select(
                'c.CustomerID',
                'c.CustomerName',
                'o.OrderID',
                'o.OrderDate',
                'o.TotalAmount'
                )

In [ ]:
OrderDetailsAgg_df = df_OrderDetails.alias('od').join(
    df_Products.alias('p'),
    (F.col('od.ProductID') == F.col('p.ProductID')) & 
    (F.col('p.IsActive') == 1) &
    (F.col('p.IsDiscontinued') == 0)
    ).join(
        df_Inventory.alias('i'),
        (F.col('od.ProductID') == F.col('i.ProductID')) & (F.col('i.StockLevel') > 0),
        how = 'full'
        ).filter(
            F.col('od.OrderID').isNotNull
            ).groupBy(
                'od.OrderID'
                ).agg(
                    F.countDistinct('od.ProductID').alias('DistinctProducts'),
                    F.sum(F.col('od.Quantity') * F.col('od.UnitPrice')).alias('OrderValue'),
                    F.max('p.Category').alias('TopCategory')
                    ).select(
                        'od.OrderID',
                        'DistinctProducts',
                        'OrderValue',
                        'TopCategory'
                        )

In [ ]:
HighSpenders_df = CustomerActivity_df.alias('ca').join(
    OrderDetailsAgg_df.alias('oda'),
    F.col('oda.OrderID') == F.col('ca.OrderID'),
    how = 'left'
    ).groupBy(
        'ca.CustomerID',
        'ca.CustomerName'
        ).agg(
            F.avg('ca.TotalAmount').alias('AvgTop3OrderAmount'),
            F.sum('oda.OrderValue').alias('TotalTopOrderValue'),
            F.max('oda.TopCategory').alias('FrequentCategory')
            ).select(
                'ca.CustomerID',
                'ca.CustomerName',
                'AvgTop3OrderAmount',
                'FrequentCategory',
                'TotalTopOrderValue',
                F.when(
                    F.col('TotalTopOrderValue') > 20000, 'Platinum' 
                    )
                .when(
                    ((F.col('TotalTopOrderValue') >= 10000) & (F.col('TotalTopOrderValue') <= 20000)), 'Gold' 
                    )
                .otherwise('Silver').alias('Tier')
                ).filter(
                    F.col('AvgTop3OrderAmount') > 3000
                    ).orderBy(
                        F.col('TotalTopOrderValue').desc()
                        )

In [ ]:
WITH empdata AS (
(SELECT employee_id, first_name, manager_id, 1 AS level
FROM employee
WHERE employee_id = 1)

UNION ALL

(SELECT this.employee_id, this.first_name, this.manager_id, prior.level + 1
FROM empdata prior
INNER JOIN employee this ON this.manager_id = prior.employee_id)
)

SELECT e.employee_id, e.first_name, e.manager_id, e.level
FROM empdata e
ORDER BY e.level;

In [ ]:
result = Orders.alias('o').join(
    Customers.alias('c'),
    F.col('o.CustomerID') == F.col('c.CustomerID')
    ).join(
        OrderDetails.alias('od'),
        (F.col('od.OrderID') == F.col('o.OrderID')) & (F.col('od.Quantity') > 2),
        how = 'left'        
        ).join(
            Products.alias('p'),
            ((F.col('p.ProductID') == F.col('od.ProductID')) & (
                (
                    (F.col('p.IsActive') == 1) & (F.col('p.Discounted') == 0)
                    ) | (
                        (F.col('p.Discounted') == 1) & (F.col('p.Price') > 100)
                        )                
                    )
                ),
            how = 'left'
            ).join(
                Inventory.alias('i'),
                ((F.col('i.ProductID') == F.col('p.ProductID')) & 
                (F.col('i.StockLevel') > (
                    F.when(
                        F.col('p.Category') == 'Electronics', 10
                        ).otherwise(0)
                    ))),
                how = 'left'
                ).filter(
                    (F.col('o.OrderDate') >= F.add_months(F.current_date(), -3)) & (F.col('c.IsVIP') == 1)
                    ).select(
                        'o.OrderID',
                        'c.CustomerName',
                        'p.ProductName',
                        'od.Quantity',
                        'i.StockLevel'
                        )
                